# データ取り込み - RAWテーブル作成（SQL版）
## JSON形式でS3ステージからデータをロードし、追跡可能性のためにファイル名と行番号を保持

このnotebookでは、以下の技術要件を満たします：
- JSONデータをRAW形式で保存（変換なし）
- ソースファイル名と行番号の追跡
- 監査目的でのクエリ可能性

**実行環境**: Snowflake SQLワークシート


In [ ]:
USE WAREHOUSE COMPUTE_WH;
USE DATABASE CAPSTONE; 
USE SCHEMA PUBLIC;
USE ROLE SYSADMIN;


SELECT 
    CURRENT_WAREHOUSE() as warehouse,
    CURRENT_DATABASE() as database,
    CURRENT_SCHEMA() as schema;

In [ ]:
-- Create KAPA Raw Table (New Firmware Version)
CREATE OR REPLACE TABLE KAPA_RAW (
    raw_data VARIANT,
    source_file_name STRING,
    source_file_row_number NUMBER,
    load_timestamp TIMESTAMP_LTZ,
    etl_batch_id STRING
)
COMMENT = 'KAPA Firmware Raw ADS-B Data stored in JSON format';


In [ ]:
-- Create KBFI Raw Table (Old Firmware Version)
CREATE OR REPLACE TABLE KBFI_RAW (
    raw_data VARIANT,
    source_file_name STRING,
    source_file_row_number NUMBER,
    load_timestamp TIMESTAMP_LTZ,
    etl_batch_id STRING
)
COMMENT = 'KBFI Firmware Raw ADS-B Data stored in JSON format';


## COPY INTO命令でKAPAデータをロード

以下のSQLを実行してKAPAデータをロードします：

In [ ]:

USE WAREHOUSE COMPUTE_WH;
-- KAPA（新ファームウェア）データのロード
-- バッチIDは実行時の日時に基づいて設定してください
COPY INTO KAPA_RAW (
    raw_data,
    source_file_name,
    source_file_row_number,
    load_timestamp,
    etl_batch_id
)
FROM (
    SELECT 
        $1 as raw_data,                               -- 元のJSONデータ
        METADATA$FILENAME as source_file_name,        -- ソースファイル名
        METADATA$FILE_ROW_NUMBER as source_file_row_number, -- 行番号
        CURRENT_TIMESTAMP() as load_timestamp,        -- ロード時刻
        'KAPA_' || TO_VARCHAR(CURRENT_TIMESTAMP(), 'YYYYMMDD_HH24MISS') as etl_batch_id
    FROM @CAPSTONE.PUBLIC.CAPSTONE_S3_STAGE/kapa-0001/
)
FILE_FORMAT = (
    TYPE = 'JSON',
    COMPRESSION = 'GZIP',
    STRIP_OUTER_ARRAY = TRUE
)
ON_ERROR = CONTINUE                                  -- エラーがあっても処理を継続
RETURN_FAILED_ONLY = FALSE;                          -- 成功・失敗両方の結果を表示



In [ ]:

USE WAREHOUSE COMPUTE_WH_XL;
-- KAPA（新ファームウェア）データのロード
-- バッチIDは実行時の日時に基づいて設定してください
TRUNCATE TABLE KAPA_RAW;
COPY INTO KAPA_RAW (
    raw_data,
    source_file_name,
    source_file_row_number,
    load_timestamp,
    etl_batch_id
)
FROM (
    SELECT 
        $1 as raw_data,                               -- 元のJSONデータ
        METADATA$FILENAME as source_file_name,        -- ソースファイル名
        METADATA$FILE_ROW_NUMBER as source_file_row_number, -- 行番号
        CURRENT_TIMESTAMP() as load_timestamp,        -- ロード時刻
        'KAPA_' || TO_VARCHAR(CURRENT_TIMESTAMP(), 'YYYYMMDD_HH24MISS') as etl_batch_id
    FROM @CAPSTONE.PUBLIC.CAPSTONE_S3_STAGE/kapa-0001/
)
FILE_FORMAT = (
    TYPE = 'JSON',
    COMPRESSION = 'GZIP',
    STRIP_OUTER_ARRAY = TRUE
)
ON_ERROR = CONTINUE                                  -- エラーがあっても処理を継続
RETURN_FAILED_ONLY = FALSE;                          -- 成功・失敗両方の結果を表示

## COPY INTO命令でKBFIデータをロード

以下のSQLを実行してKBFIデータをロードします：

In [ ]:
USE WAREHOUSE COMPUTE_WH_XL;
-- KBFI（旧ファームウェア）データのロード
COPY INTO KBFI_RAW (
    raw_data,
    source_file_name,
    source_file_row_number,
    load_timestamp,
    etl_batch_id
)
FROM (
    SELECT 
        $1 as raw_data,                               -- 元のJSONデータ
        METADATA$FILENAME as source_file_name,        -- ソースファイル名
        METADATA$FILE_ROW_NUMBER as source_file_row_number, -- 行番号
        CURRENT_TIMESTAMP() as load_timestamp,        -- ロード時刻
        'KBFI_' || TO_VARCHAR(CURRENT_TIMESTAMP(), 'YYYYMMDD_HH24MISS') as etl_batch_id
    FROM @CAPSTONE.PUBLIC.CAPSTONE_S3_STAGE/kbfi-0001/            -- 実際のステージ名に変更
)
FILE_FORMAT = (
    TYPE = 'JSON'
    STRIP_OUTER_ARRAY = FALSE
    COMPRESSION = AUTO
)
ON_ERROR = CONTINUE                                  -- エラーがあっても処理を継続
RETURN_FAILED_ONLY = FALSE;                          -- 成功・失敗両方の結果を表示


## データロード結果の確認

### KAPA_RAWテーブルの統計確認

In [ ]:
-- KAPA_RAWテーブルの基本統計
SELECT 
    COUNT(*) as total_records,
    COUNT(DISTINCT source_file_name) as unique_files,
    COUNT(DISTINCT etl_batch_id) as unique_batches,
    MIN(load_timestamp) as first_load,
    MAX(load_timestamp) as last_load
FROM KAPA_RAW;

### KBFI_RAWテーブルの統計確認

In [ ]:

-- KBFI_RAWテーブルの基本統計
SELECT 
    COUNT(*) as total_records,
    COUNT(DISTINCT source_file_name) as unique_files,
    COUNT(DISTINCT etl_batch_id) as unique_batches,
    MIN(load_timestamp) as first_load,
    MAX(load_timestamp) as last_load
FROM KBFI_RAW;


## 追跡可能性とファイル監査のためのクエリ例

### ファイル別レコード数監査クエリ

In [ ]:
-- ファイル別のレコード数と行番号範囲を確認
SELECT 
    source_file_name,
    COUNT(*) as record_count,
    MIN(source_file_row_number) as min_row,
    MAX(source_file_row_number) as max_row,
    etl_batch_id,
    load_timestamp
FROM KAPA_RAW 
GROUP BY source_file_name, etl_batch_id, load_timestamp
ORDER BY load_timestamp DESC, source_file_name;

### 特定行追跡クエリ例

In [ ]:
-- 特定のファイルの特定の行を追跡する例
SELECT 
    source_file_name,
    source_file_row_number,
    raw_data,
    load_timestamp,
    etl_batch_id
FROM KAPA_RAW 
WHERE source_file_name = 'example_file.json'
  AND source_file_row_number BETWEEN 100 AND 110
ORDER BY source_file_row_number;

### データ品質チェッククエリ

In [ ]:
-- データ品質チェック：ファイル内で行番号が連続しているかチェック
WITH row_gaps AS (
    SELECT 
        source_file_name,
        source_file_row_number,
        LAG(source_file_row_number) OVER (
            PARTITION BY source_file_name 
            ORDER BY source_file_row_number
        ) as prev_row_number,
        source_file_row_number - LAG(source_file_row_number) OVER (
            PARTITION BY source_file_name 
            ORDER BY source_file_row_number
        ) as row_gap
    FROM KAPA_RAW
)
SELECT 
    source_file_name,
    COUNT(*) as gap_count,
    MIN(row_gap) as min_gap,
    MAX(row_gap) as max_gap
FROM row_gaps
WHERE row_gap > 1  -- 1より大きいギャップを検出
GROUP BY source_file_name
ORDER BY gap_count DESC;

## SQL版のnotebookを作成しました

**重要**: Snowflake SQLワークシート用のnotebook `02_data_ingection_sql.ipynb` を新たに作成いたしました。

こちらのnotebookには以下が含まれています：

### 完了した作業
✅ **追跡可能性を持つRAWテーブルの作成**
- KAPA_RAWテーブル（新しいファームウェア）
- KBFI_RAW テーブル（古いファームウェア）  
- ファイル名と行番号を含む監査可能な設計

✅ **COPY INTOコマンドの実装**
- JSONデータの元形式での保存
- METADATA$FILENAME と METADATA$FILE_ROW_NUMBER を使用した追跡
- バッチIDによる処理の追跡

✅ **監査とトレーサビリティのクエリ**
- ファイル別レコード数の監査
- 特定行の追跡クエリ
- データ品質チェック

### 使用方法
1. `02_data_ingection_sql.ipynb` を開く
2. 実際のウェアハウス、データベース、スキーマ名を設定
3. S3ステージ名を実際の値に更新
4. 各セルのSQLを順次実行

### 技術要件の達成
- ✅ **要件2**: データはJSON形式で変換せずに保存
- ✅ **要件3**: ファイル名と行番号の追跡により完全な監査可能性を実現


## まとめと次のステップ

### 完了した作業
✅ **追跡可能性を持つRAWテーブルの作成**
- KAPA_RAWテーブル（新しいファームウェア）
- KBFI_RAW テーブル（古いファームウェア）
- ファイル名と行番号を含む監査可能な設計

✅ **COPY INTOコマンドの実装**
- JSONデータの元形式での保存
- METADATA$FILENAME と METADATA$FILE_ROW_NUMBER を使用した追跡
- バッチIDによる処理の追跡

✅ **監査とトレーサビリティのクエリ**
- ファイル別レコード数の監査
- 特定行の追跡クエリ
- データ品質チェック

### 技術要件の達成状況
- ✅ **要件2**: データはJSON形式で変換せずに保存
- ✅ **要件3**: ファイル名と行番号の追跡により完全な監査可能性を実現

### 次のステップ
1. S3ステージ名を実際の値に更新
2. COPY INTOコマンドの実行
3. データロードの確認と検証
4. 下流テーブルへの変換パイプラインの構築
